<a href="https://colab.research.google.com/github/Giffy/MongoDB_PyMongo_Tutorial/blob/master/2_1_Mobile_coverage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example 2 - Mobile phone coverage

#1. System setup

##1.1 Install Pymongo

In [0]:
!pip install pymongo==3.7.2   # Install PyMongo
!pip install folium geopandas    # Install folium for map visualization

##1.2 Connect to Google Drive

In [0]:
# Check is Link to Drive is OK
google = !if [ -d 'GDrive/' ]; then echo "1" ; else echo "0"; fi
if (google[0] is '0' ):
  from google.colab import drive
  drive.mount('/content/GDrive/')
!if [ -d 'GDrive/' ]; then echo "Connection to Google drive successful" ; else echo "Error to connect to Google drive"; fi

#2. Import libraries and set database

##2.1 Import libraries

In [0]:
import pymongo                            # Library to access MongoDB
from pymongo import MongoClient           # Imports MongoClient 
import pandas as pd                       # Library to work with dataframes
import folium                             # Library to create the map
from folium import plugins                # plugin to create a heatmap
#import matplotlib.pyplot as plt  

##2.2 Connect to database

In [0]:
# uri (uniform resource identifier) defines the connection parameters 
uri = 'localhost:27017'
# start client to connect to MongoDB server 
client = MongoClient( uri )

##2.3 Download database and import dataset into mongodb
There are a lot of datasets available if you want to practice Mongo with BigData.

Data has been downloaded from: [Dades Obertes de la Generalitat de Catalunya](http://governobert.gencat.cat/ca/dades_obertes/)

Original dataset includes mobile phone coverage data in Catalunya, have 11M records and weights 1.7Gb.

url = "https://analisi.transparenciacatalunya.cat/api/views/g9ma-vbt8/rows.tsv?accessType=DOWNLOAD&bom=true"


In [0]:
# Download dataset for exercise
url = "https://raw.githubusercontent.com/Giffy/Personal_dataset_repository/master/telefoniaBCN.csv.zip"
!wget $url
!unzip telefoniaBCN.csv.zip

##2.4 Load data to database

In [0]:
!mongodb-linux-x86_64-debian71-3.0.15/bin/mongoimport --db catalunya --collection mobileBCN --type csv --file telefoniaBCN.csv --drop --headerline

#!Export data as CSV
#!mongodb-linux-x86_64-debian71-3.0.15/bin/mongoexport --db catalunya --collection mobileBCN --type csv --out telefoniaBCN.csv --fields timestamp_,Senyal,OperadorDesc_,net_type,Activitat,NOM_MUNI,Year,Month,Hour,Carrier,weekday,Lat,Lng


## mongoimport:
##    --db            Database_Name
##    --collections   Collection_Name
##    --type          json (default)
##                    csv (for comma-separated values file)
##                    tsv (tab separated values)
##    --jsonArray     to import JSON files in a JSON array
##    --file          Filename 
##    --fields        To define field name
##    --headerline    To define first rows (column name) as field name
##
##    --help


# imported 382378

#3. Data overview

In [0]:
databases = client.list_database_names()                # Show database names
db = client.catalunya                                   # Set the database to work in
collections = db.list_collection_names()                # List the collections available 
collection = db.mobileBCN                               # Set collection 
print ("Databases: " + str(databases) + "\nCollections: " + str(collections))      

In [0]:
list( collection.find().limit(1) )

In [0]:
collection.count_documents({'_id' : {'$exists' : 1} })

In [0]:
print ( collection.distinct('Carrier') )
print ( collection.distinct('Month') )
print ( collection.distinct('Activitat') )

In [0]:
collection.create_index([( 'Carrier', pymongo.ASCENDING)])        # Creates an index for Carrier
collection.create_index([( 'Month', pymongo.ASCENDING)])          # Creates an index for Month
collection.create_index([( 'Activitat', pymongo.ASCENDING)])      # Creates an index for Activitat
collection.create_index([( 'net_type', pymongo.ASCENDING)])       # Creates an index for net_type

collection.index_information()                                    # Shows the existing indexes

# Queries to Database

## Carrier Market share in Barcelona 

In [0]:
carrier_list = collection.distinct('Carrier')

total = 0                                  # Contador del total de documentos
for carrier in carrier_list:
  total_operador = collection.count_documents({'Carrier': carrier })
  total += total_operador
  print ( carrier + ': ' + str(total_operador))
print ( total)

In [0]:
pipeline = [{ '$group':{ 
                  '_id' : { 'Carrier' : '$Carrier'}
                  ,'Carrier' : {'$max' : '$Carrier'}
                  ,'Total' : {'$sum' : 1}
                  }
            },
            { '$project': 
                  { '_id' : 0,
                    'Carrier' : 1,
                    'Total' : 1}
            },
            { '$sort': {'Total' : -1}
            }
            ]
query = list ( collection.aggregate( pipeline ) ) 
query 

In [0]:
df = pd.DataFrame(query)
df

In [0]:
df = df.set_index('Carrier')
df.plot(kind='bar');

## Heatmap

In [0]:
list( collection.find({'Carrier' : 'movistar'}, {'_id', 'Carrier', 'Lat', 'Lng'}).limit(2) )

In [0]:
pipeline = [{ '$group':{ 
                  '_id' : { 'lat': '$Lat' , 'lon' : '$Lng', 'hour' : '$Hour'}
#                  '_id' : { 'Carrier' : '$Carrier', 'lat': '$Lat' , 'lon' : '$Lng'}
#                  ,'Carrier' : {'$max' : '$Carrier'}
                  ,'lat' : {'$max' : '$Lat'}
                  ,'lon' : {'$max' : '$Lng'}
                  ,'hour': {'$max' : '$Hour'}
                  ,'Total' : {'$sum' : 1}
                  }
            },
            { '$match':{
                  'hour' : 9
#                  ,'Carrier' : 'movistar'
                  }
            },
            { '$project': 
                  { '_id' : 0,
                    'lat' : 1,
                    'lon' : 1,
#                    'Carrier' : 1,
                    'hour' : 1,
                    'Total' : 1}
            },
            { '$sort': {'Total' : -1}
            }
            ]

query = list ( collection.aggregate( pipeline ) ) 
df = pd.DataFrame(query)
query[0]

In [0]:
print ( "lat " + str ( float(df.loc[0]['lat']) ) )
print ( "lng " + str ( float(df.loc[0]['lon']) ) )
print ( "totals " + str ( float(df.loc[0]['Total']) ))
# extract lat, lon, and magnitude for the folium heatmap
longitud  = len( df )
lats = []
lngs = []
totals = []
for i in range ( longitud ):
    lats.append ( float(df.loc[i]['lat']) )
    lngs.append ( float(df.loc[i]['lon']) )
    totals.append(float(df.loc[i]['Total']) )

In [0]:
center_lat = 41.388
center_lon = 2.159

locationmap = folium.Map(location=[ center_lat , center_lon ], zoom_start=13, width=800, height=600 )
locationmap.add_child(plugins.HeatMap(zip(lats, lngs, totals), radius = 12))

locationmap

In [0]:
pipeline = [{ '$group':{ 
                  '_id' : { 'lat': '$Lat' , 'lon' : '$Lng', 'hour' : '$Hour'}
#                  '_id' : { 'Carrier' : '$Carrier', 'lat': '$Lat' , 'lon' : '$Lng'}
#                  ,'Carrier' : {'$max' : '$Carrier'}
                  ,'lat' : {'$max' : '$Lat'}
                  ,'lon' : {'$max' : '$Lng'}
                  ,'hour': {'$max' : '$Hour'}
                  ,'Total' : {'$sum' : 1}
                  }
            },
            { '$match':{
                  'hour' : 21
#                  ,'Carrier' : 'movistar'
                  }
            },
            { '$project': 
                  { '_id' : 0,
                    'lat' : 1,
                    'lon' : 1,
#                    'Carrier' : 1,
                    'hour' : 1,
                    'Total' : 1}
            },
            { '$sort': {'Total' : -1}
            }
            ]

query = list ( collection.aggregate( pipeline ) ) 
df = pd.DataFrame(query)
query[0]

In [0]:
print ( "lat " + str ( float(df.loc[0]['lat']) ) )
print ( "lng " + str ( float(df.loc[0]['lon']) ) )
print ( "totals " + str ( float(df.loc[0]['Total']) ))
# extract lat, lon, and magnitude for the folium heatmap
longitud  = len( df )
lats = []
lngs = []
totals = []
for i in range ( longitud ):
    lats.append ( float(df.loc[i]['lat']) )
    lngs.append ( float(df.loc[i]['lon']) )
    totals.append(float(df.loc[i]['Total']) )

In [0]:
center_lat = 41.388
center_lon = 2.159

locationmap = folium.Map(location=[ center_lat , center_lon ], zoom_start=13, width=800, height=600 )
locationmap.add_child(plugins.HeatMap(zip(lats, lngs, totals), radius = 12))

locationmap

## Results

### Barcelona at 9 AM
<img src='https://raw.githubusercontent.com/Giffy/MongoDB_PyMongo_Tutorial/master/resources/Bcn9AM.png' width="600"/>
### Barcelona at 9 PM
<img src='https://raw.githubusercontent.com/Giffy/MongoDB_PyMongo_Tutorial/master/resources/Bcn9PM.png' width="600"/>